# Dependencies

In [1]:
!pip install --quiet flashtext
!pip install --quiet git+https://github.com/boudinfl/pke.git
!pip install --quiet transformers
!pip install --quiet sentencepiece

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.8 MB/s eta 0:00:00


In [7]:
import json
import requests
import string
import re
import nltk
import string
import itertools
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')
import pke
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import traceback
from nltk.tokenize import sent_tokenize
from flashtext import KeywordProcessor

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


# Text Preprocessing

In [8]:
def tokenize_sentences(text):
    sentences = sent_tokenize(text)
    sentences = [sentence.strip() for sentence in sentences if len(sentence) > 20]
    return sentences

In [127]:
import textwrap

text = """ Process scheduling is a fundamental concept in operating systems that ensures efficient utilization of the CPU by managing the execution of multiple processes. In a multitasking environment, the CPU must allocate its time among various processes, balancing responsiveness, throughput, and fairness. The operating system achieves this through a scheduler, which determines the order in which processes are executed based on predefined algorithms.

There are three primary types of process schedulers: Long-term scheduler, Short-term scheduler, and Medium-term scheduler. The long-term scheduler selects processes from the job queue and loads them into the ready queue, controlling the degree of multiprogramming. The short-term scheduler, also known as the CPU scheduler, decides which process gets the CPU next and for how long. Lastly, the medium-term scheduler temporarily removes processes from memory to reduce the load and improve system performance, a process known as swapping.

Scheduling algorithms, such as First-Come, First-Served (FCFS), Shortest Job Next (SJN), Round Robin (RR), and Priority Scheduling, define how processes are prioritized. For instance, Round Robin ensures fairness by allocating a fixed time slice to each process, while Priority Scheduling executes processes based on their assigned priority.

Efficient process scheduling reduces CPU idle time, minimizes waiting time, and ensures that high-priority tasks are completed promptly. It plays a crucial role in maintaining system stability and ensuring optimal performance across diverse computing environments."""

wrapper = textwrap.TextWrapper(width=150)
word_list = wrapper.wrap(text=text)
for element in word_list:
  print(element)

 Process scheduling is a fundamental concept in operating systems that ensures efficient utilization of the CPU by managing the execution of multiple
processes. In a multitasking environment, the CPU must allocate its time among various processes, balancing responsiveness, throughput, and fairness.
The operating system achieves this through a scheduler, which determines the order in which processes are executed based on predefined algorithms.
There are three primary types of process schedulers: Long-term scheduler, Short-term scheduler, and Medium-term scheduler. The long-term scheduler
selects processes from the job queue and loads them into the ready queue, controlling the degree of multiprogramming. The short-term scheduler, also
known as the CPU scheduler, decides which process gets the CPU next and for how long. Lastly, the medium-term scheduler temporarily removes processes
from memory to reduce the load and improve system performance, a process known as swapping.  Scheduling alg

In [128]:
def get_keywords(text):
    out=[]
    try:
        extractor = pke.unsupervised.YAKE()
        extractor.load_document(input=text,language='en')
        grammar = r"""
                NP:
                    {<NOUN|PROPN>+}
            """
        extractor.ngram_selection(n=1)
        extractor.grammar_selection(grammar=grammar)
        extractor.candidate_selection(n=1)
        extractor.candidate_weighting(window=3,
                                      use_stems=False)
        keyphrases = extractor.get_n_best(n=30)
        for val in keyphrases:
            out.append(val[0])
    except:
        out = []
        traceback.print_exc()

    return out

keywords = get_keywords(text)[:8]
print ("keywords: ",keywords)

keywords:  ['cpu', 'scheduler', 'processes', 'scheduling', 'process', 'priority', 'time', 'round']


In [129]:
sentences = tokenize_sentences(text)
print (sentences)

['Process scheduling is a fundamental concept in operating systems that ensures efficient utilization of the CPU by managing the execution of multiple processes.', 'In a multitasking environment, the CPU must allocate its time among various processes, balancing responsiveness, throughput, and fairness.', 'The operating system achieves this through a scheduler, which determines the order in which processes are executed based on predefined algorithms.', 'There are three primary types of process schedulers: Long-term scheduler, Short-term scheduler, and Medium-term scheduler.', 'The long-term scheduler selects processes from the job queue and loads them into the ready queue, controlling the degree of multiprogramming.', 'The short-term scheduler, also known as the CPU scheduler, decides which process gets the CPU next and for how long.', 'Lastly, the medium-term scheduler temporarily removes processes from memory to reduce the load and improve system performance, a process known as swappi

In [130]:
from pprint import pprint
def get_sentences_for_keyword(keywords, sentences):
    keyword_processor = KeywordProcessor()
    keyword_sentences = {}
    for word in keywords:
        keyword_sentences[word] = []
        keyword_processor.add_keyword(word)
    for sentence in sentences:
        keywords_found = keyword_processor.extract_keywords(sentence)
        for key in keywords_found:
            keyword_sentences[key].append(sentence)

    for key in keyword_sentences.keys():
        values = keyword_sentences[key]
        values = sorted(values, key=len, reverse=False)
        keyword_sentences[key] = values
    return keyword_sentences

keyword_sentence_mapping = get_sentences_for_keyword(keywords, sentences)
pprint (keyword_sentence_mapping)

{'cpu': ['The short-term scheduler, also known as the CPU scheduler, decides '
         'which process gets the CPU next and for how long.',
         'The short-term scheduler, also known as the CPU scheduler, decides '
         'which process gets the CPU next and for how long.',
         'Efficient process scheduling reduces CPU idle time, minimizes '
         'waiting time, and ensures that high-priority tasks are completed '
         'promptly.',
         'In a multitasking environment, the CPU must allocate its time among '
         'various processes, balancing responsiveness, throughput, and '
         'fairness.',
         'Process scheduling is a fundamental concept in operating systems '
         'that ensures efficient utilization of the CPU by managing the '
         'execution of multiple processes.'],
 'priority': ['Efficient process scheduling reduces CPU idle time, minimizes '
              'waiting time, and ensures that high-priority tasks are '
              'complet

# BERT WSD Model

In [15]:
!pip install --quiet transformers

In [131]:
import torch
import math
from transformers import BertModel, BertConfig, BertPreTrainedModel, BertTokenizer

class BertWSD(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)

        self.bert = BertModel(config)
        self.dropout = torch.nn.Dropout(config.hidden_dropout_prob)

        self.ranking_linear = torch.nn.Linear(config.hidden_size, 1)

        self.init_weights()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_dir = "/content/BERT-WSD"


model = BertWSD.from_pretrained(model_dir)
tokenizer = BertTokenizer.from_pretrained(model_dir)
tokenizer.added_tokens_encoder['[TGT]'] = 100


if '[TGT]' not in tokenizer.additional_special_tokens:
    tokenizer.add_special_tokens({'additional_special_tokens': ['[TGT]']})

model.resize_token_embeddings(len(tokenizer))
model.to(device)
model.eval()

BertWSD(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30524, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affin

In [158]:
import csv
import os
from collections import namedtuple
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
import torch
import re
import time
import torch
from tabulate import tabulate
from torch.nn.functional import softmax
from tqdm import tqdm
from transformers import BertTokenizer


GlossSelectionRecord = namedtuple("GlossSelectionRecord", ["guid", "sentence", "sense_keys", "glosses", "targets"])
BertInput = namedtuple("BertInput", ["input_ids", "input_mask", "segment_ids", "label_id"])

MAX_SEQ_LENGTH = 128

def _create_features_from_records(records, max_seq_length, tokenizer, cls_token_at_end=False, pad_on_left=False,
                                  cls_token='[CLS]', sep_token='[SEP]', pad_token=0,
                                  sequence_a_segment_id=0, sequence_b_segment_id=1,
                                  cls_token_segment_id=1, pad_token_segment_id=0,
                                  mask_padding_with_zero=True, disable_progress_bar=False):
    features = []
    for record in tqdm(records, disable=disable_progress_bar):
        tokens_a = tokenizer.tokenize(record.sentence)

        sequences = [(gloss, 1 if i in record.targets else 0) for i, gloss in enumerate(record.glosses)]

        pairs = []
        for seq, label in sequences:
            tokens_b = tokenizer.tokenize(seq)
            _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
            tokens = tokens_a + [sep_token]
            segment_ids = [sequence_a_segment_id] * len(tokens)

            tokens += tokens_b + [sep_token]
            segment_ids += [sequence_b_segment_id] * (len(tokens_b) + 1)

            if cls_token_at_end:
                tokens = tokens + [cls_token]
                segment_ids = segment_ids + [cls_token_segment_id]
            else:
                tokens = [cls_token] + tokens
                segment_ids = [cls_token_segment_id] + segment_ids

            input_ids = tokenizer.convert_tokens_to_ids(tokens)
            input_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

            padding_length = max_seq_length - len(input_ids)
            if pad_on_left:
                input_ids = ([pad_token] * padding_length) + input_ids
                input_mask = ([0 if mask_padding_with_zero else 1] * padding_length) + input_mask
                segment_ids = ([pad_token_segment_id] * padding_length) + segment_ids
            else:
                input_ids = input_ids + ([pad_token] * padding_length)
                input_mask = input_mask + ([0 if mask_padding_with_zero else 1] * padding_length)
                segment_ids = segment_ids + ([pad_token_segment_id] * padding_length)

            assert len(input_ids) == max_seq_length
            assert len(input_mask) == max_seq_length
            assert len(segment_ids) == max_seq_length

            pairs.append(
                BertInput(input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids, label_id=label)
            )

        features.append(pairs)

    return features


def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [159]:
from pprint import pprint
nltk.download('omw-1.4')
sentence = "Mark's favourite game is **Cricket**."

sentence_for_bert = sentence.replace("**"," [TGT] ")
sentence_for_bert = " ".join(sentence_for_bert.split())

print (sentence_for_bert)

re_result = re.search(r"\[TGT\](.*)\[TGT\]", sentence_for_bert)
if re_result is None:
    print("\nIncorrect input format. Please try again.")

ambiguous_word = re_result.group(1).strip()

print ("Word: ",ambiguous_word)


results = dict()

# wn_pos = wn.NOUN
# for i, synset in enumerate(set(wn.synsets(ambiguous_word, pos=wn_pos))):
for i, synset in enumerate(set(wn.synsets(ambiguous_word))):
    results[synset] =  synset.definition()

pprint (results)

sense_keys=[]
definitions=[]
for sense_key, definition in results.items():
    sense_keys.append(sense_key)
    definitions.append(definition)


print (sense_keys)
print (definitions)


record = GlossSelectionRecord("test", sentence_for_bert, sense_keys, definitions, [-1])

Mark's favourite game is [TGT] Cricket [TGT] .
Word:  Cricket
{Synset('cricket.n.01'): 'leaping insect; male makes chirping noises by '
                         'rubbing the forewings together',
 Synset('cricket.n.02'): 'a game played with a ball and bat by two teams of 11 '
                         'players; teams take turns trying to score runs',
 Synset('cricket.v.01'): 'play cricket'}
[Synset('cricket.n.02'), Synset('cricket.n.01'), Synset('cricket.v.01')]
['a game played with a ball and bat by two teams of 11 players; teams take turns trying to score runs', 'leaping insect; male makes chirping noises by rubbing the forewings together', 'play cricket']


[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [160]:
model.config.vocab_size

30524

In [161]:
features = _create_features_from_records([record], MAX_SEQ_LENGTH, tokenizer,
                                          cls_token=tokenizer.cls_token,
                                          sep_token=tokenizer.sep_token,
                                          cls_token_segment_id=1,
                                          pad_token_segment_id=0,
                                          disable_progress_bar=True)[0]

print (len(features))

for ftr in features:
  print (tokenizer.convert_ids_to_tokens(ftr.input_ids))

3
['[CLS]', 'mark', "'", 's', 'favourite', 'game', 'is', '[TGT]', 'cricket', '[TGT]', '.', '[SEP]', 'a', 'game', 'played', 'with', 'a', 'ball', 'and', 'bat', 'by', 'two', 'teams', 'of', '11', 'players', ';', 'teams', 'take', 'turns', 'trying', 'to', 'score', 'runs', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PA

In [162]:
print(f"Vocabulary size: {len(tokenizer)}")
print("Sample input_ids:", features[0].input_ids)
assert max(features[0].input_ids) < len(tokenizer), "Input IDs exceed tokenizer vocabulary size!"


Vocabulary size: 30524
Sample input_ids: [101, 2928, 1005, 1055, 8837, 2208, 2003, 30523, 4533, 30523, 1012, 102, 1037, 2208, 2209, 2007, 1037, 3608, 1998, 7151, 2011, 2048, 2780, 1997, 2340, 2867, 1025, 2780, 2202, 4332, 2667, 2000, 3556, 3216, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [163]:
with torch.no_grad():
      logits = torch.zeros(len(definitions), dtype=torch.double).to(device)
      for i, bert_input in list(enumerate(features)):
          logits[i] = model.ranking_linear(
              model.bert(
                  input_ids=torch.tensor(bert_input.input_ids, dtype=torch.long).unsqueeze(0).to(device),
                  attention_mask=torch.tensor(bert_input.input_mask, dtype=torch.long).unsqueeze(0).to(device),
                  token_type_ids=torch.tensor(bert_input.segment_ids, dtype=torch.long).unsqueeze(0).to(device)
              )[1]
          )
      scores = softmax(logits, dim=0)

      preds = (sorted(zip(sense_keys, definitions, scores), key=lambda x: x[-1], reverse=True))

print ("\n")
for pred in preds:
  print (pred)
sense = preds[0][0]
meaning = preds[0][1]

print ("\nMost appropriate sense: ",sense)
print ("Most appropriate meaning: ",meaning)



(Synset('cricket.v.01'), 'play cricket', tensor(0.5838, dtype=torch.float64))
(Synset('cricket.n.02'), 'a game played with a ball and bat by two teams of 11 players; teams take turns trying to score runs', tensor(0.4162, dtype=torch.float64))
(Synset('cricket.n.01'), 'leaping insect; male makes chirping noises by rubbing the forewings together', tensor(1.9887e-05, dtype=torch.float64))

Most appropriate sense:  Synset('cricket.v.01')
Most appropriate meaning:  play cricket


In [164]:
def get_sense(sent):
  re_result = re.search(r"\[TGT\](.*)\[TGT\]", sent)
  if re_result is None:
      print("\nIncorrect input format. Please try again.")

  ambiguous_word = re_result.group(1).strip()
  results = dict()

  for i, synset in enumerate(set(wn.synsets(ambiguous_word))):
      results[synset] =  synset.definition()

  if len(results) ==0:
    return None

  sense_keys=[]
  definitions=[]
  for sense_key, definition in results.items():
      sense_keys.append(sense_key)
      definitions.append(definition)

  record = GlossSelectionRecord("test", sent, sense_keys, definitions, [-1])

  features = _create_features_from_records([record], MAX_SEQ_LENGTH, tokenizer,
                                            cls_token=tokenizer.cls_token,
                                            sep_token=tokenizer.sep_token,
                                            cls_token_segment_id=1,
                                            pad_token_segment_id=0,
                                            disable_progress_bar=True)[0]

  with torch.no_grad():
      logits = torch.zeros(len(definitions), dtype=torch.double).to(device)
      for i, bert_input in list(enumerate(features)):
          logits[i] = model.ranking_linear(
              model.bert(
                  input_ids=torch.tensor(bert_input.input_ids, dtype=torch.long).unsqueeze(0).to(device),
                  attention_mask=torch.tensor(bert_input.input_mask, dtype=torch.long).unsqueeze(0).to(device),
                  token_type_ids=torch.tensor(bert_input.segment_ids, dtype=torch.long).unsqueeze(0).to(device)
              )[1]
          )
      scores = softmax(logits, dim=0)

      preds = (sorted(zip(sense_keys, definitions, scores), key=lambda x: x[-1], reverse=True))

  sense = preds[0][0]
  meaning = preds[0][1]
  return sense


In [165]:
import statistics
from statistics import mode
import re

def get_synsets_for_word (word):
  return set(wn.synsets(word))

keyword_best_sense = {}

for keyword in  keyword_sentence_mapping:
  print ("\n\n")
  print("Original word: ",keyword)
  try:
    identified_synsets=get_synsets_for_word(keyword)
  except:
    continue
  for synset in identified_synsets:
    print (synset,"   ",synset.definition())
  top_3_sentences = keyword_sentence_mapping[keyword][:3]
  best_senses=[]
  for sent in top_3_sentences:
    insensitive_keyword = re.compile(re.escape(keyword), re.IGNORECASE)
    modified_sentence = insensitive_keyword.sub(" [TGT] "+keyword+" [TGT] ", sent,count=1)
    modified_sentence = " ".join(modified_sentence.split())
    print ("modified sentence ",modified_sentence)
    best_sense = get_sense(modified_sentence)
    best_senses.append(best_sense)
  best_sense = mode(best_senses)
  print ("Best sense: ",best_sense)
  defn = best_sense.definition()
  print (defn)
  keyword_best_sense [keyword] = defn




Original word:  cpu
Synset('central_processing_unit.n.01')     (computer science) the part of a computer (a microprocessor chip) that does most of the data processing
modified sentence  The short-term scheduler, also known as the [TGT] cpu [TGT] scheduler, decides which process gets the CPU next and for how long.
modified sentence  The short-term scheduler, also known as the [TGT] cpu [TGT] scheduler, decides which process gets the CPU next and for how long.
modified sentence  Efficient process scheduling reduces [TGT] cpu [TGT] idle time, minimizes waiting time, and ensures that high-priority tasks are completed promptly.
Best sense:  Synset('central_processing_unit.n.01')
(computer science) the part of a computer (a microprocessor chip) that does most of the data processing



Original word:  scheduler
Synset('scheduler.n.01')     computer hardware that arranges jobs to be done by the computer in an appropriate order
modified sentence  The short-term [TGT] scheduler [TGT] , also k

In [166]:
pprint (keyword_best_sense)

{'cpu': '(computer science) the part of a computer (a microprocessor chip) '
        'that does most of the data processing',
 'priority': 'status established in order of importance or urgency',
 'process': 'deal with in a routine way',
 'processes': 'perform mathematical and logical operations on (data) according '
              'to programmed instructions in order to obtain the required '
              'information',
 'round': 'the usual activities in your day',
 'scheduler': 'computer hardware that arranges jobs to be done by the computer '
              'in an appropriate order',
 'scheduling': 'plan for an activity or event',
 'time': 'a period of time considered as a resource under your control and '
         'sufficient to accomplish something'}


In [167]:
import random
from prettytable import PrettyTable
x = PrettyTable()
all_keywords= list(keyword_best_sense.keys())
all_definitions = list(keyword_best_sense.values())
random.shuffle(all_keywords)
random.shuffle(all_definitions)
print (all_keywords)
print (all_definitions)

['processes', 'cpu', 'scheduling', 'priority', 'round', 'process', 'scheduler', 'time']
['status established in order of importance or urgency', '(computer science) the part of a computer (a microprocessor chip) that does most of the data processing', 'plan for an activity or event', 'a period of time considered as a resource under your control and sufficient to accomplish something', 'perform mathematical and logical operations on (data) according to programmed instructions in order to obtain the required information', 'computer hardware that arranges jobs to be done by the computer in an appropriate order', 'deal with in a routine way', 'the usual activities in your day']


In [168]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

x.field_names=['Word', "Definition"]
for word,defn in zip(all_keywords,all_definitions):
  x.add_row([word,defn])

printmd("**Match the following words to their correct meanings.**")

print (x)

**Match the following words to their correct meanings.**

+------------+----------------------------------------------------------------------------------------------------------------------------------------+
|    Word    |                                                               Definition                                                               |
+------------+----------------------------------------------------------------------------------------------------------------------------------------+
| processes  |                                          status established in order of importance or urgency                                          |
|    cpu     |                (computer science) the part of a computer (a microprocessor chip) that does most of the data processing                 |
| scheduling |                                                     plan for an activity or event                                                      |
|  priority  |                  a period of time considered as a resource under your con